In [ ]:
!pip install selenium
# Selenium requires a driver to interface with the chosen browser. 
# Chrome driver https://sites.google.com/a/chromium.org/chromedriver/downloads

## Setup
https://developer.xero.com/documentation/oauth2/auth-flow

In [1]:
#setup client_id (application id)
client_id = ''
#setup client_secret (application secret)
client_secret = '' 
#setup redirect_url (application redirect url)
redirect_url = 'https://xero.com/'
#setup scope to access, whole list - https://developer.xero.com/documentation/oauth2/scopes
scope_list = ['offline_access', 
              'accounting.transactions.read', 'accounting.journals.read', 'accounting.settings.read', 
              'payroll.employees.read','payroll.payruns.read', 'payroll.payslip.read','assets.read'] 

In [2]:
import json
import requests
from selenium import webdriver
#import webbrowser
import base64
import pandas as pd
import datetime
import time
import os
import getpass
import keyring

assert client_id, 'client_id is empty'
assert client_secret, 'client_secret is empty'
assert redirect_url, 'redirect_url is empty'

scope = ' '.join(scope_list)

b64_id_secret = base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')).decode('utf-8')

def FirstAuth():
    # Send a user to authorize your app
    auth_url = ('''https://login.xero.com/identity/connect/authorize?''' +
                '''response_type=code''' +
                '''&client_id=''' + client_id +
                '''&redirect_uri=''' + redirect_url +
                '''&scope=''' + scope)
    #####################################################
    # - Interaction required
    #webbrowser.open_new(auth_url)  
    # Users are redirected back with a code  
    #print()
    #auth_res_url = input('What is the response URL? ')
    ####################################################
    
    #####################################################
    # + Without Interaction      
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(auth_url)
    auth_res_url = auth_url
    #waiting while there will be "code" in the browser's URL
    while auth_res_url.find('code=') == -1:
        time.sleep(1) # 1 second sleep
        auth_res_url = driver.current_url        
  ######################################################  
    
    start_number = auth_res_url.find('code=') + len('code=')
    end_number = auth_res_url.find('&scope')
    auth_code = auth_res_url[start_number:end_number]
    #print(auth_code+'\n')
    
    # Exchange the code
    exchange_code_url = 'https://identity.xero.com/connect/token'
    response = requests.post(exchange_code_url, 
                            headers = {
                                'Authorization': 'Basic ' + b64_id_secret
                            },
                            data = {
                                'grant_type': 'authorization_code',
                                'code': auth_code,
                                'redirect_uri': redirect_url
                            })
    json_response = response.json()
    #print(json_response + '\n')
     
    # Return the tokens
    return {'access' : json_response['access_token'], 'refresh' : json_response['refresh_token']}


# Check the full set of tenants that user have been authorized to access
def GetTenants(access_token):
    connections_url = 'https://api.xero.com/connections'
    response = requests.get(connections_url,
                           headers = {
                               'Authorization': 'Bearer ' + access_token,
                               'Content-Type': 'application/json'
                           })
    json_response = response.json()
    
    #print(json_response)

    return([tenants['tenantId'] for tenants in json_response])

# Refreshing access tokens
def RefreshTokens(refresh_token):
    token_refresh_url = 'https://identity.xero.com/connect/token'
    response = requests.post(token_refresh_url,
                            headers = {
                                'Authorization' : 'Basic ' + b64_id_secret,
                                'Content-Type': 'application/x-www-form-urlencoded'
                            },
                            data = {
                                'grant_type' : 'refresh_token',
                                'refresh_token' : refresh_token
                            })
    json_response = response.json()
    #print(json_response)
    
    keyring.set_password('refresh_token', getpass.getuser(), json_response['refresh_token'])
    keyring.set_password('refresh_token_date', getpass.getuser(), str(datetime.datetime.now().timestamp()))
    
    return {'access' : json_response['access_token'], 'refresh' : json_response['refresh_token']}

# Data API
def GetData(api_url, tokens, tenant_id=''):
  
    if tenant_id == '':    
        tenant_id = XeroTenants(tokens['access'])[0]#very first tenant
   
    #print('xero_tenant_id = ' + xero_tenant_id)
     
    #https://developer.xero.com/documentation/oauth2/auth-flow
    data_type = api_url[api_url.rfind('/')+1:]
    last_page = False
    page = 1
    ret = list()
    while not last_page: 
        response = requests.get(api_url+'?page='+str(page),
                               headers = {
                                   'Authorization': 'Bearer ' + tokens['access'],
                                   'Xero-tenant-id': tenant_id,
                                   'Accept': 'application/json'
                               })
       
        #print(response)
        
        if response.status_code!=200:
            print('Bad server response for \'{}\' : {}'.format(data_type, response))
            if response.status_code not in (401, 403, 404):
                print('Retrieved JSON:\n{}'.format(response.json()))
            return ret;    
        else:
            json_response = response.json()
           
            data_response = json_response[data_type]

            ret = ret + data_response

            if len(data_response) < 100:
                last_page=True
            else:
                page+=1
        
        
    return ret


In [3]:
Accounts_URL = 'https://api.xero.com/api.xro/2.0/Accounts'
BankTransactions_URL = 'https://api.xero.com/api.xro/2.0/BankTransactions'
TrackingCategories_URL = 'https://api.xero.com/api.xro/2.0/TrackingCategories'
Invoices_URL = 'https://api.xero.com/api.xro/2.0/Invoices'
Users_URL = 'https://api.xero.com/api.xro/2.0/Users'
Employees_URL = 'https://api.xero.com/api.xro/2.0/Employees'

refresh_token_age = -1
tokens = dict()

token_date = keyring.get_password('refresh_token_date', getpass.getuser())

if token_date is not None:
    token_date = datetime.datetime.fromtimestamp(int(float(token_date)))
    #print('refresh_token_date = {}'.format(token_date))
    current_date = datetime.datetime.now() 
    refresh_token_age = (current_date - token_date).seconds/3600
    #print('refresh_token_age = {}'.format(refresh_token_age))
    
if (refresh_token_age < 0) | (refresh_token_age > 59):    
    tokens = FirstAuth()
else:
    tokens = {'refresh' : keyring.get_password("refresh_token", getpass.getuser())}

tokens = RefreshTokens(tokens['refresh'])
tenant_id = GetTenants(tokens['access'])[0]

# Get Data

df_accounts = pd.DataFrame(GetData(Accounts_URL, tokens, tenant_id))

df_bank_transactions = pd.DataFrame(GetData(BankTransactions_URL, tokens, tenant_id))

df_tracking_categories = pd.DataFrame(GetData(TrackingCategories_URL, tokens, tenant_id))

df_invoices = pd.DataFrame(GetData(Invoices_URL, tokens, tenant_id))

df_users = pd.DataFrame(GetData(Users_URL, tokens, tenant_id))

df_employees = pd.DataFrame(GetData(Employees_URL, tokens, tenant_id))
      

In [4]:
df_accounts

,AccountID,Code,Name,Status,Type,TaxType,Class,EnablePaymentsToAccount,ShowInExpenseClaims,BankAccountNumber,BankAccountType,CurrencyCode,ReportingCode,ReportingCodeName,HasAttachments,UpdatedDateUTC,AddToWatchlist,Description,SystemAccount
0,ac993f75-035b-433c-82e0-7b7a2d40802c,090,Business Bank Account,ACTIVE,BANK,NONE,ASSET,False,False,1201020345678000,BANK,NZD,ASS,Assets,False,/Date(1602989731783+0000)/,False,NaN,NaN
1,ebd06280-af70-4bed-97c6-7451a454ad85,091,Business Savings Account,ACTIVE,BANK,NONE,ASSET,False,False,0209087654321050,BANK,NZD,ASS,Assets,False,/Date(1602989731783+0000)/,False,NaN,NaN
2,7d05a53d-613d-4eb2-a2fc-dcb6adb80b80,200,Sales,ACTIVE,REVENUE,OUTPUT2,REVENUE,False,False,NaN,,NaN,REV.TRA,Trading revenue,False,/Date(1602989731783+0000)/,True,Income from any normal business activity,
3,ba3f61ed-2992-4468-bd4d-fd3ee2cb16c9,260,Other Revenue,ACTIVE,REVENUE,OUTPUT2,REVENUE,False,False,NaN,,NaN,REV.OTH,Other revenue,False,/Date(1602989731783+0000)/,False,Any other income that does not relate to norma...,
4,32865dbc-8b1e-4a76-8f5c-3579cbb3520b,270,Interest Income,ACTIVE,REVENUE,NONE,REVENUE,False,False,NaN,,NaN,REV.INV.INT,Interest,False,/Date(1602989731783+0000)/,False,Gross interest income (i.e. before deducting R...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,d08883ef-50e9-4ae4-b9d3-a5c4984658a2,877,Tracking Transfers,ACTIVE,CURRLIAB,NONE,LIABILITY,False,False,NaN,,NaN,LIA.CUR,Current Liabilities,False,/Date(1602989731783+0000)/,False,Transfers between tracking categories,TRACKINGTRANSFERS
58,641b46db-2b69-4cb2-be3d-610ee7e2217f,900,Loan,ACTIVE,TERMLIAB,NONE,LIABILITY,False,False,NaN,,NaN,LIA.NCL.LOA,Loans,False,/Date(1602989731783+0000)/,False,Money that has been borrowed from a creditor,
59,4f4528dd-c8a3-4e31-8a45-e2e4221de791,960,Retained Earnings,ACTIVE,EQUITY,NONE,EQUITY,False,False,NaN,,NaN,EQU.RET,Retained earnings/Accumulated funds,False,/Date(1602989731783+0000)/,False,Do not Use,RETAINEDEARNINGS
60,f409f07f-c72d-4478-b188-8563deb9e1a6,970,Owner A Funds Introduced,ACTIVE,EQUITY,NONE,EQUITY,True,False,NaN,,NaN,LIA.NCL.ADV,Proprietor/shareholder advance accounts,False,/Date(1602989731783+0000)/,False,Funds contributed by the owner,


In [5]:
df_bank_transactions

,BankTransactionID,BankAccount,Type,Reference,IsReconciled,HasAttachments,Contact,DateString,Date,Status,LineAmountTypes,LineItems,SubTotal,TotalTax,Total,UpdatedDateUTC,CurrencyCode
0,63d47b99-e1ef-4b46-84db-034f2205f8fb,{'AccountID': 'ac993f75-035b-433c-82e0-7b7a2d4...,SPEND,Sub 098801,False,False,{'ContactID': '6d42f03b-181f-43e3-93fb-2025c01...,2020-10-14T00:00:00,/Date(1602633600000+0000)/,AUTHORISED,Inclusive,"[{'Description': 'Subscription', 'UnitAmount':...",43.39,6.51,49.90,/Date(1290384000000+0000)/,NZD
1,303aa035-16dd-45f1-9710-310dc654a7c9,{'AccountID': 'ac993f75-035b-433c-82e0-7b7a2d4...,RECEIVE,Sub 098801,False,False,{'ContactID': '6d42f03b-181f-43e3-93fb-2025c01...,2020-10-16T00:00:00,/Date(1602806400000+0000)/,AUTHORISED,Inclusive,[{'Description': 'Long-term subscriber discoun...,30.43,4.57,35.00,/Date(1290556800000+0000)/,NZD
2,1e1eaf36-cb33-4914-9c23-78dac6508f04,{'AccountID': 'ac993f75-035b-433c-82e0-7b7a2d4...,SPEND,Fee,True,False,{'ContactID': '2b6d2035-ded0-45a7-b2ad-4b01c78...,2020-07-23T00:00:00,/Date(1595462400000+0000)/,AUTHORISED,Inclusive,"[{'Description': 'Bank fee', 'UnitAmount': 15....",15.00,0.00,15.00,/Date(1290614258523+0000)/,NZD
3,10b4de96-eefa-4dae-9eb8-d611081c97d0,{'AccountID': 'ac993f75-035b-433c-82e0-7b7a2d4...,SPEND,Eft,True,False,{'ContactID': '873c07a8-511a-420d-a664-a01a508...,2020-07-28T00:00:00,/Date(1595894400000+0000)/,AUTHORISED,Inclusive,"[{'Description': 'Misc stationery', 'UnitAmoun...",18.88,2.83,21.71,/Date(1290614269007+0000)/,NZD
4,8b043e8b-2df2-41ce-b5a7-437f2851376f,{'AccountID': 'ac993f75-035b-433c-82e0-7b7a2d4...,SPEND,,True,False,{'ContactID': 'c37099f9-b207-4c0a-84de-a635eea...,2020-07-30T00:00:00,/Date(1596067200000+0000)/,AUTHORISED,Inclusive,"[{'Description': 'Team coffees', 'UnitAmount':...",13.91,2.09,16.00,/Date(1290614270600+0000)/,NZD
5,cbd5def7-d950-4fd9-affa-e9e0fa3753ff,{'AccountID': 'ac993f75-035b-433c-82e0-7b7a2d4...,SPEND,,True,False,{'ContactID': '2a7f9bc5-d0fe-40bf-834d-6c7be44...,2020-08-02T00:00:00,/Date(1596326400000+0000)/,AUTHORISED,Inclusive,[{'Description': 'Misc kitchen supplies for of...,56.70,8.50,65.20,/Date(1290614272347+0000)/,NZD
6,140cf65f-062e-4616-9a94-9eb066e3f601,{'AccountID': 'ac993f75-035b-433c-82e0-7b7a2d4...,SPEND,Gift,True,False,{'ContactID': 'ae136984-5d9a-4025-a045-bf4dc42...,2020-08-06T00:00:00,/Date(1596672000000+0000)/,AUTHORISED,Inclusive,"[{'Description': 'Bouquet for client', 'UnitAm...",43.48,6.52,50.00,/Date(1290614277917+0000)/,NZD
7,5970fa02-c8a6-412f-ac86-cee98c2d4438,{'AccountID': 'ac993f75-035b-433c-82e0-7b7a2d4...,SPEND,,True,False,{'ContactID': 'baf81bcb-eebf-4b8a-977d-95b11f5...,2020-08-09T00:00:00,/Date(1596931200000+0000)/,AUTHORISED,Inclusive,"[{'Description': 'Team coffee', 'UnitAmount': ...",13.57,2.03,15.60,/Date(1290614279787+0000)/,NZD
8,e6248e6d-1daf-425f-95fe-5c5f273ff6b1,{'AccountID': 'ac993f75-035b-433c-82e0-7b7a2d4...,SPEND,Chq 409,True,False,{'ContactID': '8c555bc8-8098-43cd-bfa1-acd5573...,2020-08-09T00:00:00,/Date(1596931200000+0000)/,AUTHORISED,Inclusive,"[{'Description': 'Monthly carpark', 'UnitAmoun...",129.13,19.37,148.50,/Date(1290614291083+0000)/,NZD
9,02ec32ad-fc69-464d-9655-f04b1f0a50f8,{'AccountID': 'ac993f75-035b-433c-82e0-7b7a2d4...,SPEND,Fee,True,False,{'ContactID': '2b6d2035-ded0-45a7-b2ad-4b01c78...,2020-08-22T00:00:00,/Date(1598054400000+0000)/,AUTHORISED,Inclusive,"[{'Description': 'Bank fee', 'UnitAmount': 15....",15.00,0.00,15.00,/Date(1290804224810+0000)/,NZD


In [6]:
df_tracking_categories

,Name,Status,TrackingCategoryID,Options
0,Region,ACTIVE,351953c4-8127-4009-88c3-f9cd8c9cbe9f,[{'TrackingOptionID': 'ce205173-7387-4651-9726...


In [7]:
df_invoices

,Type,InvoiceID,InvoiceNumber,Reference,Payments,CreditNotes,Prepayments,Overpayments,AmountDue,AmountPaid,...,LineItems,SubTotal,TotalTax,Total,UpdatedDateUTC,CurrencyCode,RepeatingInvoiceID,FullyPaidOnDate,SentToContact,BrandingThemeID
0,ACCPAY,22b3fab4-ef56-4d70-a110-a7cc3c1a26cd,Elec.,,[],[],[],[],0.00,0.0,...,"[{'Description': 'Monthly electricity', 'UnitA...",77.39,11.61,89.00,/Date(1289566516347+0000)/,NZD,NaN,NaN,NaN,NaN
1,ACCPAY,96988e67-ecf9-466d-bfbf-0afa1725a649,RPT445-1,,[{'PaymentID': '0a0ef7ee-7b91-46fa-8136-c4cc62...,[],[],[],0.00,90.0,...,"[{'Description': 'Monthly electricity', 'UnitA...",78.26,11.74,90.00,/Date(1289572094303+0000)/,NZD,ad3550bc-1ae0-45c0-a782-48c6d2061127,/Date(1569974400000+0000)/,NaN,NaN
2,ACCPAY,6e65ea34-ccd7-49e7-b80a-b3feec4c1c48,RPT445-1,,[{'PaymentID': '603b8347-d833-4e65-abf9-1f4656...,[],[],[],0.00,95.5,...,"[{'Description': 'Monthly electricity', 'UnitA...",83.04,12.46,95.50,/Date(1289572118670+0000)/,NZD,ad3550bc-1ae0-45c0-a782-48c6d2061127,/Date(1572480000000+0000)/,NaN,NaN
3,ACCPAY,33f11d54-2105-4041-b77b-9097578e7c9d,RPT445-1,,[{'PaymentID': '285d0c41-5e16-45d2-90b6-673c91...,[],[],[],0.00,97.0,...,"[{'Description': 'Monthly electricity', 'UnitA...",84.35,12.65,97.00,/Date(1289579146917+0000)/,NZD,ad3550bc-1ae0-45c0-a782-48c6d2061127,/Date(1574899200000+0000)/,NaN,NaN
4,ACCPAY,ac1fc284-2e69-46b6-95b3-17949c65bb42,RPT445-1,,[{'PaymentID': 'e41cab17-9785-45f2-88d4-6aad48...,[],[],[],0.00,92.0,...,"[{'Description': 'Monthly electricity', 'UnitA...",80.00,12.00,92.00,/Date(1289579167027+0000)/,NZD,ad3550bc-1ae0-45c0-a782-48c6d2061127,/Date(1577664000000+0000)/,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,ACCREC,69a49503-69fb-4da0-9ac0-69e421c6e50c,INV-0034,P/O 9711,[],[],[],[],650.00,0.0,...,[{'Description': 'Development work - develper ...,565.22,84.78,650.00,/Date(1466787093510+0000)/,NZD,NaN,NaN,True,7889a0ac-262a-40e3-8a63-9a769b1a18af
75,ACCREC,0be4a903-e493-4c3e-b001-6b68d23524f6,INV-0035,P/O CRM08-12,[],[],[],[],6187.50,0.0,...,[{'Description': 'Onsite project management fo...,5380.43,807.07,6187.50,/Date(1466787115637+0000)/,NZD,NaN,NaN,True,7889a0ac-262a-40e3-8a63-9a769b1a18af
76,ACCREC,33a66615-d092-413a-bd80-54609f7a755d,INV-0028,Book,[],[],[],[],19.95,0.0,...,"[{'ItemCode': 'BOOK', 'Description': ''Fish ou...",17.35,2.60,19.95,/Date(1466787133887+0000)/,NZD,NaN,NaN,True,7889a0ac-262a-40e3-8a63-9a769b1a18af
77,ACCREC,1fcc5819-1b6c-437b-b984-ae2e5c8ed46b,INV-0023,P/O 9711,[],[],[],[],250.00,0.0,...,"[{'ItemCode': 'PR-BR', 'Description': 'Project...",217.39,32.61,250.00,/Date(1466787170870+0000)/,NZD,NaN,NaN,True,7889a0ac-262a-40e3-8a63-9a769b1a18af


### If case if needed to delete saved passwords

In [ ]:
keyring.delete_password("refresh_token", getpass.getuser())
keyring.delete_password("refresh_token_date", getpass.getuser())